In [ ]:
result = pd.read_csv("total_hit_rate.csv")

In [ ]:
p_result = result.rename(columns = {"attacker_name" : "character_name"})
p_result = p_result.iloc[:,1:2] 
p_result

In [ ]:
# 회복/부스팅 아이템 리스트 생성
get_items = ['Item_Heal_MedKit', 'Item_Heal_FirstAid_C', 'Item_Heal_Bandage_C', 'Item_Boost_AdrenalineSyringe_C', 'Item_Boost_PainKiller_C', 'Item_Boost_EnergyDrink_C', 'Item_Weapon_Molotov_C', 'Item_Weapon_SmokeBomb_C', 'Item_Weapon_FlashBang_C', 'Item_Weapon_Grenade_C']

# 데이터를 담을 데이터프레임 생성
get_data = pd.DataFrame()

# 전체 매치에 대해 반복
for i in range(18):
    # 텔레메트리 JSON 데이터 가져오기
    telemetry_url = df_mat['telemetry_url'][i]
    rs_tele_event = requests.get(telemetry_url, headers=header)
    telemetry_json = rs_tele_event.json()

    # JSON 데이터를 데이터프레임으로 변환 후, LogMatchStart와 LogMatchEnd 사이의 데이터 추출
    df_log = pd.DataFrame(telemetry_json)
    df_log = df_log.loc[df_log[df_log['_T'] == 'LogMatchStart'].index[0]:df_log[df_log['_T'] == 'LogMatchEnd'].index[0]]

    # LogItemPickup 이벤트 추출 후, A 팀의 아이템 획득자만 필터링
    item_log = df_log[(df_log['_T'] == 'LogItemPickup') | (df_log['_T'] == "LogItemPickupFromLootBox")]
    df_item_log = item_log[item_log['character'].astype(str).str.contains('')].reset_index(drop=True)

    # character_name 컬럼을 만들어주기 위해 NaN으로 초기화
    df_item_log['character_name'] = np.NaN 

    # character_name 컬럼에 데이터 추가
    for j in range(len(df_item_log)):
        df_item_log.loc[j, 'character_name'] = df_item_log[df_item_log['character'].astype(str).str.contains("")]['character'][j]['name']

    # 해당 팀 각 선수들이 얻은 각 아이템 개수 구하기
    team_item_data = p_result
    for item in get_items:
        item_count = df_item_log[df_item_log['item'].astype(str).str.contains(item)].groupby('character_name').count()['item']
        item_count = pd.DataFrame(item_count).reset_index()
        item_count.rename(columns={"item": item}, inplace=True)
        team_item_data = pd.merge(team_item_data, item_count, on='character_name', how='outer')
        team_item_data['match_id'] = i + 1
    get_data = pd.concat([get_data, team_item_data], ignore_index=True)

# 결과 출력
get_data


In [ ]:
t_result = result.iloc[:,1:2]
t_result

In [ ]:
# Throwable 아이템 리스트
Throwable_items = ['Item_Weapon_Molotov_C', 'Item_Weapon_SmokeBomb_C', 'Item_Weapon_Grenade_C', 'Item_Weapon_FlashBang_C']

# 결과 데이터프레임 초기화
Throwable_data = pd.DataFrame()

# 18개의 경기 데이터에 대해 반복문 실행
for i in range(18):
    # telemetry JSON 데이터 가져오기
    telemetry_url = df_mat['telemetry_url'][i]
    rs_tele_event = requests.get(telemetry_url, headers=header)
    telemetry_json = rs_tele_event.json()

    # JSON 데이터를 데이터프레임으로 변환 후, LogMatchStart와 LogMatchEnd 사이의 데이터 추출
    df_log = pd.DataFrame(telemetry_json)
    df_log = df_log.loc[df_log[df_log['_T'] == 'LogMatchStart'].index[0]:df_log[df_log['_T'] == 'LogMatchEnd'].index[0]]

    # LogPlayerUseThrowable 이벤트 추출 후, A 팀의 아이템 사용자만 필터링
    item_log = df_log[df_log['_T'] == 'LogPlayerUseThrowable']
    df_item_log = item_log[item_log['attacker'].astype(str).str.contains('')].reset_index(drop=True)

    # attacker_name 컬럼을 만들어주기 위해 NaN으로 초기화
    df_item_log['attacker_name'] = np.NaN 

    # attacker_name 컬럼에 데이터 추가
    for j in range(len(df_item_log)):
        df_item_log.loc[j,'attacker_name'] = df_item_log[df_item_log['attacker'].astype(str).str.contains('')]['attacker'][j]['name']

    # 해당 팀 각 선수들이 사용한 각 throwable 아이템 개수 구하기
    team_item_data = t_result
    for item in Throwable_items:
        item_count = df_item_log[df_item_log['weapon'].astype(str).str.contains(item)].groupby('attacker_name').count()['weapon']
        item_count = pd.DataFrame(item_count).reset_index()
        item_count.rename(columns={"weapon": item}, inplace=True)
        team_item_data = pd.merge(team_item_data, item_count, on='attacker_name', how='outer')
        team_item_data['match_id'] = i + 1
    Throwable_data = pd.concat([Throwable_data, team_item_data], ignore_index=True)
    
# 결과 데이터프레임 출력
Throwable_data

In [ ]:
h_result = result.iloc[:,1:2]
h_result = h_result.rename(columns = {"attacker_name" : "character_name"})
h_result

In [ ]:
# 사용할 아이템 리스트 정의
heal_boost_use_items = ['Item_Heal_MedKit', 'Item_Heal_FirstAid_C', 'Item_Heal_Bandage_C', 'Item_Boost_AdrenalineSyringe_C', 'Item_Boost_PainKiller_C', 'Item_Boost_EnergyDrink_C']

# 결과를 저장할 데이터프레임 초기화
heal_boost_use_data = pd.DataFrame()

# 18개의 매치에 대해 데이터 수집
for i in range(18):
    # telemetry JSON 데이터 가져오기
    telemetry_url = df_mat['telemetry_url'][i]
    rs_tele_event = requests.get(telemetry_url, headers=header)
    telemetry_json = rs_tele_event.json()

    # JSON 데이터를 데이터프레임으로 변환 후, LogMatchStart와 LogMatchEnd 사이의 데이터 추출
    df_log = pd.DataFrame(telemetry_json)
    df_log = df_log.loc[df_log[df_log['_T'] == 'LogMatchStart'].index[0]:df_log[df_log['_T'] == 'LogMatchEnd'].index[0]]

    # LogItemPickup 이벤트 추출 후, 팀의 아이템 사용자만 필터링
    item_log = df_log[df_log['_T'] == 'LogItemUse']
    df_item_log = item_log[item_log['character'].astype(str).str.contains('')].reset_index(drop=True)

    # character_name 컬럼을 만들어주기 위해 NaN으로 초기화
    df_item_log['character_name'] = np.NaN 

    # character_name 컬럼에 데이터 추가
    for j in range(len(df_item_log)):
        df_item_log.loc[j, 'character_name'] = df_item_log[df_item_log['character'].astype(str).str.contains('')]['character'][j]['name']

    # 해당 팀 각 선수들이 사용한 각 아이템 개수 구하기
    team_item_data = h_result
    for item in heal_boost_use_items:
        item_count = df_item_log[df_item_log['item'].astype(str).str.contains(item)].groupby('character_name').count()['item']
        item_count = pd.DataFrame(item_count).reset_index()
        item_count.rename(columns={"item": item}, inplace=True)
        team_item_data = pd.merge(team_item_data, item_count, on='character_name', how='outer')
        team_item_data['match_id'] = i + 1
    heal_boost_use_data = pd.concat([heal_boost_use_data, team_item_data], ignore_index=True)

# 결과 데이터프레임 출력
heal_boost_use_data
